In [17]:
import pandas as pd
import xgboost as xgb
import numpy as np
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
molecule_matrix = np.array(pd.read_csv('Dataset/Molecule_Matrix.csv'))
label = np.array(pd.read_csv('Dataset/label.csv')).T[0]
moleculeCID = np.array(pd.read_csv('Dataset/moleculeCID.csv')).T[0]

In [3]:
print(molecule_matrix.shape)
print(len(label))
print(len(moleculeCID))
print(Counter(label))

(305385, 158)
305385
305385
Counter({0: 304501, 1: 884})


#### Down sampling majority first,  inactive:active = 5:1

In [4]:
ratio = {0:4420,1:884}
rus = RandomUnderSampler(random_state=8584096,ratio=ratio)
x_resample,y_resample = rus.fit_sample(molecule_matrix,label)

In [5]:
# split the train and test
x_train,x_test,y_train,y_test = train_test_split(x_resample,y_resample,test_size = 0.3)

In [98]:
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 3086, 1: 626})
Counter({0: 1334, 1: 258})


#### load numpy into DMatrix

In [47]:
train = xgb.DMatrix(x_train,label=y_train)
test = xgb.DMatrix(x_test,label=y_test)

In [95]:
param = {'lambda':0.8,'max_depth':5,'scale_pos_weight':5,'gamma':8,'objective':'binary:logistic','eval_metric':'auc','eta':0.4}
num_round = 10
watchlist = [(test,'eval'),(train,'train')]

In [96]:
bst = xgb.train(dtrain=train,params=param,num_boost_round=num_round,evals=watchlist)

[0]	eval-auc:0.523507	train-auc:0.631773
[1]	eval-auc:0.541813	train-auc:0.734095
[2]	eval-auc:0.530046	train-auc:0.787516
[3]	eval-auc:0.519529	train-auc:0.800178
[4]	eval-auc:0.525802	train-auc:0.816931
[5]	eval-auc:0.527448	train-auc:0.825031
[6]	eval-auc:0.511211	train-auc:0.859444
[7]	eval-auc:0.513623	train-auc:0.871295
[8]	eval-auc:0.514613	train-auc:0.88362
[9]	eval-auc:0.506641	train-auc:0.900185


In [38]:
preds = bst.predict(test)

In [39]:
label = test.get_label()

In [40]:
t = [1 if i>0.6 else 0 for i in preds]

In [41]:
np.mean(t!=label)

0.28266331658291455

In [42]:
confusion_matrix(y_true=label,y_pred=t)

array([[1094,  240],
       [ 210,   48]])

In [43]:
tn,fp,fn,tp = confusion_matrix(y_true=label,y_pred=t).ravel()

In [99]:
# print('Precision=', tp/(tp+fp))

In [100]:
# print('Recall:',tp/(tp+fn))

In [101]:
# print('False negative:',fn/(tp+fn))